En primer lugar preparamos el entorno importando las librerías que se van a utilizar

In [1]:
# Manejo de datos
import pandas as pd
import numpy as np

# Texto y similitud
from sklearn.metrics.pairwise import cosine_similarity

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades
import os
import pickle
from collections import Counter, defaultdict

# OpenAI embeddings
from openai import OpenAI
from dotenv import load_dotenv


En primer lugar realizamos la limpieza del csv resultante del formulario que se encuentra en la carpeta llamada "data"

In [13]:
# corrected relative path (ensure the 'data' folder is one level up from the notebook)
registros = pd.read_csv("../data/Graph-Match _ CIS 2026-I(1-5).csv", encoding="latin-1")

In [15]:
#eliminamos las columnas que están vacías
registros = registros.dropna(axis=1, how="all")
#eliminamos las filas que no aceptaron participar en la actividad
registros = registros[registros["¿Aceptas participar en esta actividad?"] != "No"]
registros.head(5)

,ID,Start time,Completion time,Email,Name,¿Aceptas participar en esta actividad?,¿ Cómo te llamas ?,¿ Cómo te pueden contactar ? Puedes escribir tu usuario de Instagram o tu número de Whatsapp.\n Esta información solo será visible para las personas con las que tengas match.,¿ Cuantos años tienes ?,Da una descripción de ti !! ¿ Cómo eres física y emocionalmente ? ?????????????? (Esto lo van a leer tus match),...,¿ Estas buscando algo serio ? ??????,¿ Qué tan extrovertido/a eres ?,¿ Qué tan emocional eres ?,¿ Que tan romántico/a eres ?,¿ Eres la cucharita grande o la cucharita pequeña ?,¿ Cómo expresas tu love lenguaje (lenguaje de amor) ? ?????,¿ Como te gusta recibir amor ? ?????,¿ Con qué género te identificas ?,¿Con qué tipo de personas te gustaría conectar?,Describe tu tipo ideal !! (Física y emocionalmente)
0,1,2/1/26 17:35:45,2/1/26 17:38:14,anonymous,NaN,NaN,TestUsuario,Test,NaN,Test,...,Si,5.0,5.0,NaN,NaN,NaN,NaN,Mujer,Hombres,Test
1,2,2/3/26 13:32:13,2/3/26 13:35:53,ambanos@uninorte.edu.co,ANGELICA MARIA BAÑOS PALLARES,NaN,Angélica,AngieIsTalking,18-20,Meow meow meow meow meow meow meow,...,Si,3.0,4.0,4.0,Pequeña,Tiempo de calidad;Actos de servicio;Palabras;,Tiempo de calidad;Actos de servicio;Contacto f...,Mujer,Hombres,Un femboy bien gótico ????????
2,3,2/5/26 9:07:28,2/5/26 9:16:39,sdariana@uninorte.edu.co,DARIANA SANGUINO CUELLO,NaN,Dariana Sanguino,3052333964,18-20,"Soy blanca, delgada, mido 1,63, me considero i...",...,Si,4.0,3.0,4.0,Pequeña,Tiempo de calidad;Contacto físico;Actos de ser...,Tiempo de calidad;Regalos;Contacto físico;Acto...,Mujer,Hombres,"Que me escuche, que este pendiente de mi, que ..."
3,4,2/5/26 9:08:58,2/5/26 9:30:53,amfrias@uninorte.edu.co,Adolfo Moises Frias Pardo,NaN,Adolfo Frías,@ado_drkc,18-20,"Soy delgado, cabello oscuro ni muy corto ni mu...",...,Si,3.0,3.0,4.0,Grande,Palabras;Tiempo de calidad;Contacto físico;Reg...,Palabras;Contacto físico;Tiempo de calidad;,Hombre,Mujeres,Alguien con quién pueda hablar de lo que sea y...


A continuación convertiremos cada fila/registro en un nodo

In [ ]:
usuarios = {}

for index, row in registros.iterrows():
   user = {
       "metadata":{
           "nombre": row["nombre"],
           "contacto": row["contacto"],
           "correo": row["Email"],
           "división": row["div"],
           "rango_edad": row["edad"],
           "género": row["genero"],
           "gen_interes": row["genero_interes"],
           "algo_serio": row["algo_serio"],
           "tipo_relacion": row["tipo"]
       },
       "var_estructurales": {
           "géneros_musica": str(row["musica"]).split(";"),
           "planes_finde": str(row["finde"].split(";")),
           "tiempo_libre": str(row["freetime"].split(";")),
           "color": row["color"],
           
       },
       "escalas": {
           "nivel_extrovertido": row["extrover"],
           "nivel_emocional": row["emo"],
           "nivel_romantico": row["romantico"]
       },
       "love_and_roles": {
           "love_language_dar": row["love_language_dar"],
           "love_language_recibir": row["love_language_recibir"],
           "cucharita": row["cuchara"]
       },
       "arquetipo_visual": {
           "arquetipo_indentidad": row["me_identifico"],
           "arquetipo_ideal": row["me_gusta"],
           "pareja_favorita": row["pareja_fav"]
           
       },
       "descripcion": {
           "descripción_propia": row["mi_descripcion"],
           "descripción_tipo": row["mi_descripcion"]
       },
       "embeddings": {
           "propia": None,
           "tipo": None
       },
       "feedback": {
           "asistirá": row["asistencia"],
           "motivación": row["motivacion"]
       }
   }
   
# usamos el índice como ID único
usuarios[f"id_{index}"] = user

Una vez con cada persona convertida en un nodo, podemos pasar sus descripciones a vectores numéricos con el API de openai.

In [ ]:
load_dotenv()  # Carga las variables de entorno desde el archivo .env
openai = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY"))   


In [17]:
def generar_embedding(texto, model="text-embedding-3-small"):
    """
    Recibe un texto y devuelve su embedding como lista de floats.
    """
    if texto is None or texto.strip() == "":
        return None

    response = openai.embeddings.create(
        model=model,
        input=texto
    )

    return response.data[0].embedding


In [ ]:
for user_id, user in usuarios.items():

    texto_propio = user["descripcion"]["propia"]
    texto_tipo = user["descripcion"]["tipo"]

    user["embeddings"]["propia"] = generar_embedding(texto_propio)
    user["embeddings"]["tipo"] = generar_embedding(texto_tipo)  